<!--BOOK_INFORMATION-->
<img align="left" style="padding-right:10px;" src="figures/PDSH-cover-small.png">
*This notebook contains an excerpt from the [Python Data Science Handbook](http://shop.oreilly.com/product/0636920034919.do) by Jake VanderPlas; the content is available [on GitHub](https://github.com/jakevdp/PythonDataScienceHandbook).*

*The text is released under the [CC-BY-NC-ND license](https://creativecommons.org/licenses/by-nc-nd/3.0/us/legalcode), and code is released under the [MIT license](https://opensource.org/licenses/MIT). If you find this content useful, please consider supporting the work by [buying the book](http://shop.oreilly.com/product/0636920034919.do)!*

<!--NAVIGATION-->
< [Understanding Data Types in Python](02.01-Understanding-Data-Types.ipynb) | [Contents](Index.ipynb) | [Computation on NumPy Arrays: Universal Functions](02.03-Computation-on-arrays-ufuncs.ipynb) >

# The Basics of NumPy Arrays

**Data manipulation in Python is nearly synonymous with NumPy array manipulation: even newer tools like Pandas ([Chapter 3](03.00-Introduction-to-Pandas.ipynb)) are built around the NumPy array.**
This section will present several examples of **using NumPy array manipulation to access data and subarrays, and to split, reshape, and join the arrays.**
While the types of operations shown here may seem a bit dry and pedantic, they comprise the building blocks of many other examples used throughout the book.
Get to know them well!

We'll cover a few categories of basic array manipulations here:

- *Attributes of arrays*: Determining the size, shape, memory consumption, and data types of arrays
- *Indexing of arrays*: Getting and setting the value of individual array elements
- *Slicing of arrays*: Getting and setting smaller subarrays within a larger array
- *Reshaping of arrays*: Changing the shape of a given array
- *Joining and splitting of arrays*: Combining multiple arrays into one, and splitting one array into many

## NumPy Array Attributes

First let's discuss some useful array attributes.
We'll start by defining three random arrays, a one-dimensional, two-dimensional, and three-dimensional array.
We'll use NumPy's random number generator, which we will *seed* with a set value in order to ensure that the same random arrays are generated each time this code is run:

np.random.seed(0)을 사용해 시드(seed) 값을 설정하면, 난수 생성의 시작점을 고정하게 되어 난수 생성 결과가 동일하게 재현

- 난수는 원래 무작위성을 갖지만, 시드를 고정하면 매번 동일한 결과.

In [3]:
import numpy as np
np.random.seed(41)  # 재현 가능성을 위한 시드 값 - 주석으로 처리하여 차이를 이해 

x1 = np.random.randint(10, size=6)  # 1차원 배열
x2 = np.random.randint(10, size=(3, 4))  # 2차원 배열
x3 = np.random.randint(10, size=(3, 4, 5))  # 3차원 배열
print(x2)
print(type(x2))

print(x3)

[[9 7 5 8]
 [3 3 2 6]
 [0 4 6 9]]
<class 'numpy.ndarray'>
[[[3 8 4 7 6]
  [1 5 2 1 5]
  [3 2 4 1 2]
  [3 4 5 4 5]]

 [[7 6 2 4 5]
  [8 4 6 1 4]
  [9 9 5 2 4]
  [4 4 3 0 7]]

 [[5 2 5 8 5]
  [2 0 1 7 2]
  [2 8 2 5 2]
  [8 8 2 1 0]]]


**Each array has attributes ``ndim`` (the number of dimensions), ``shape`` (the size of each dimension), and ``size`` (the total size of the array):**

In [5]:
x2.shape
x3.ndim

3

In [4]:
print("x3 ndim: ", x3.ndim)
print("x3 shape:", x3.shape)
print("x3 size: ", x3.size)

x3 ndim:  3
x3 shape: (3, 4, 5)
x3 size:  60


Another useful attribute is the ``dtype``, the data type of the array (which we discussed previously in [Understanding Data Types in Python](02.01-Understanding-Data-Types.ipynb)):

In [7]:
print("dtype:", x3.dtype)

dtype: int32


Other attributes include ``itemsize``, which lists the size (in bytes) of each array element, and ``nbytes``, which lists the total size (in bytes) of the array:

In [4]:
print("itemsize:", x3.itemsize, "bytes")
print("nbytes:", x3.nbytes, "bytes")

itemsize: 8 bytes
nbytes: 480 bytes


In general, we expect that ``nbytes`` is equal to ``itemsize`` times ``size``.

## Array Indexing: Accessing Single Elements

If you are familiar with Python's standard list indexing, indexing in NumPy will feel quite familiar.
In a one-dimensional array, the $i^{th}$ value (counting from zero) can be accessed by specifying the desired index in square brackets, just as with Python lists:

In [16]:
x1

array([5, 0, 3, 3, 7, 9])

In [6]:
x1[0]

5

In [7]:
x1[4]

7

To index from the end of the array, you can use **negative indices:**

In [8]:
x1[-1]

9

In [9]:
x1[-2]

7

In a multi-dimensional array, items can be accessed using a comma-separated tuple of indices:

In [11]:
x2

array([[3, 5, 2, 4],
       [7, 6, 8, 8],
       [1, 6, 7, 7]])

In [ ]:
x2[0, 0] # 다차원 배열의 인덱싱 

3

In [12]:
x2[2, 0]

1

In [14]:
x2[1, -1]

8

Values can also be modified using any of the above index notation:

In [14]:
x2[0, 0] = 12
x2

array([[12,  5,  2,  4],
       [ 7,  6,  8,  8],
       [ 1,  6,  7,  7]])

Keep in mind that,** unlike Python lists, NumPy arrays have a fixed type.**
This means, for example, that if you attempt to insert a floating-point value to an integer array, the value will be silently truncated. Don't be caught unaware by this behavior!

In [15]:
x1[0] = 3.14159  # 이 값의 소수점 이하는 잘릴 것이다!
x1

array([3, 0, 3, 3, 7, 9])

## Array Slicing: Accessing Subarrays

Just as we can use square brackets to access individual array elements, we can also use them to access subarrays with the *slice* notation, marked by the colon (``:``) character.
The NumPy slicing syntax follows that of the standard Python list; to access a slice of an array ``x``, use this:
``` python
x[start:stop:step]
```
If any of these are unspecified, they default to the values **``start=0``, ``stop=``*``size of dimension``*, ``step=1``.**
We'll take a look at accessing sub-arrays in one dimension and in multiple dimensions.

### One-dimensional subarrays

In [23]:
x = np.arange(10)
x

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [24]:
x[:5]  # 첫 다섯 개의 요소

array([0, 1, 2, 3, 4])

In [25]:
x[5:]  # 인덱스 5 다음의 요소들

array([5, 6, 7, 8, 9])

In [19]:
x[4:7]  # 중간 하위 배열

array([4, 5, 6])

In [27]:
x[::3]  # 하나 걸러 하나씩의 요소들로 구성된 배열

array([0, 3, 6, 9])

In [21]:
x[1::2]  # 인덱스 1로부터 시작하여 하나 걸러 하나씩 요소들로 구성된 배열

array([1, 3, 5, 7, 9])

A potentially confusing case is **when the ``step`` value is negative.
In this case, the defaults for ``start`` and ``stop`` are swapped.
This becomes a convenient way to reverse an array:**

In [28]:
x[::-1]  # 모든 요소들을 거꾸로 나열

array([9, 8, 7, 6, 5, 4, 3, 2, 1, 0])

In [29]:
x[5::-2]  # 인덱스 5부터 하나 걸러 하나씩 요소들을 거꾸로 나열

array([5, 3, 1])

### Multi-dimensional subarrays

Multi-dimensional slices work in the same way, with multiple slices separated by commas.
For example:

In [30]:
x2

array([[3, 5, 2, 4],
       [7, 6, 8, 8],
       [1, 6, 7, 7]])

In [ ]:
x2[:2, :3]  # 두 개의 행, 세 개의 열 > 2,3은 포함안됨 

array([[3, 5, 2],
       [7, 6, 8]])

In [ ]:
x2[:3, ::2]  # 0 ~ 1 행, 한 열 걸러 하나씩

array([[3, 2],
       [7, 8],
       [1, 7]])

Finally, subarray dimensions can even be reversed together:

In [ ]:
x2[::-1, ::-1] # 행과 열이 역순으로 재배치 

array([[7, 7, 6, 1],
       [8, 8, 6, 7],
       [4, 2, 5, 3]])

#### Accessing array rows and columns

One commonly needed routine is accessing of single rows or columns of an array.
This can be done by **combining indexing and slicing, using an empty slice marked by a single colon (``:``):**

In [34]:
print(x2[:, 0])  # x2의 첫 번째 열

[3 7 1]


In [36]:
print(x2[0, :])  # x2의 첫 번째 행

[3 5 2 4]


In the case of row access, the empty slice can be omitted for a more compact syntax:

In [ ]:
print(x2[0])  # x2[0, :] 와 동일 > x2가 이차원 배열인데 인덱스를 하나만 줄 때 

[12  5  2  4]


### Subarrays as no-copy views

**One important–and extremely useful–thing to know about array slices is that they return *views* rather than *copies* of the array data.**
**This is one area in which NumPy array slicing differs from Python list slicing**: in lists, slices will be copies.
Consider our two-dimensional array from before:

In [37]:
print(x2)

[[3 5 2 4]
 [7 6 8 8]
 [1 6 7 7]]


In [38]:
print(type(x2))

<class 'numpy.ndarray'>


In [39]:
l = [1,2,3,4,5,6,7,8,9]
print(l[::-1])

[9, 8, 7, 6, 5, 4, 3, 2, 1]


In [ ]:
l2= l[::-2] #리스트는 사본이다 
l2[0]= 10
print(l)

[1, 2, 3, 4, 5, 6, 7, 8, 9]


Let's extract a $2 \times 2$ subarray from this:

In [ ]:
x2_sub = x2[:2, :2]#사본이 아니라 view이다 
print(x2_sub)

[[3 5]
 [7 6]]


Now if we **modify this subarray, we'll see that the original array is changed** Observe:

In [43]:
x2_sub[0, 0] = 99
print(x2_sub)

[[99  5]
 [ 7  6]]


In [44]:
print(x2)

[[99  5  2  4]
 [ 7  6  8  8]
 [ 1  6  7  7]]


This default behavior is actually quite useful: it means that **when we work with large datasets, we can access and process pieces of these datasets without the need to copy the underlying data buffer.**

### Creating copies of arrays

Despite the nice features of array views, it is sometimes useful to instead explicitly copy the data within an array or a subarray. This can be most easily done with the **``copy()`` method:**

In [45]:
x2_sub_copy = x2[:2, :2].copy()
print(x2_sub_copy)

[[99  5]
 [ 7  6]]


If we now modify this subarray, the original array is not touched:

In [47]:
x2_sub_copy[0, 0] = 42
print(x2_sub_copy)

[[42  5]
 [ 7  6]]


In [48]:
print(x2)

[[99  5  2  4]
 [ 7  6  8  8]
 [ 1  6  7  7]]


## Reshaping of Arrays

**Another useful type of operation is reshaping of arrays.
The most flexible way of doing this is with the ``reshape`` method.**
For example, if you want to put the numbers 1 through 9 in a $3 \times 3$ grid, you can do the following:

In [50]:
grid = np.arange(1, 10)
grid=grid.reshape((3, 3))
print(grid)

[[1 2 3]
 [4 5 6]
 [7 8 9]]


Note that for this to work, the size of the initial array must match the size of the reshaped array. 
Where possible, the ``reshape`` method will use a no-copy view of the initial array, but with non-contiguous memory buffers this is not always the case.

Another common reshaping pattern is the conversion of a one-dimensional array into a two-dimensional row or column matrix.
This can be done with **the ``reshape`` method, or more easily done by making use of the ``newaxis`` keyword within a slice operation:**

In [52]:
x = np.array([1, 2, 3])
print(x)
# reshape 을 이용한 행 벡터
x.reshape((1, 3))

[1 2 3]


array([[1, 2, 3]])

In [53]:
# newaxis를 이용한 행 벡터
x[np.newaxis, :]

array([[1, 2, 3]])

In [54]:
# reshape을 이용한 열 벡터
x.reshape((3, 1))

array([[1],
       [2],
       [3]])

In [55]:
# newaxis를 이용한 열 벡터
x[:, np.newaxis]

array([[1],
       [2],
       [3]])

We will see this type of transformation often throughout the remainder of the book.

## Array Concatenation and Splitting

All of the preceding routines worked on single arrays. It's also possible to combine multiple arrays into one, and to conversely split a single array into multiple arrays. We'll take a look at those operations here.

### Concatenation of arrays

Concatenation, or joining of two arrays in NumPy, is primarily accomplished using the routines **``np.concatenate``, ``np.vstack``, and ``np.hstack``.**
``np.concatenate`` takes a tuple or list of arrays as its first argument, as we can see here:

In [4]:
x = np.array([1, 2, 3])
y = np.array([3, 2, 1])
np.concatenate([x, y])

array([1, 2, 3, 3, 2, 1])

You can also concatenate more than two arrays at once:

In [5]:
z = [99, 99, 99]
print(np.concatenate([x, z,y]))

[ 1  2  3 99 99 99  3  2  1]


It can also be used for two-dimensional arrays:

In [7]:
grid = np.array([[1, 2, 3],
                 [4, 5, 6]])

In [8]:
# 첫 번째 축을 따라 연결
np.concatenate([grid, grid])

array([[1, 2, 3],
       [4, 5, 6],
       [1, 2, 3],
       [4, 5, 6]])

np.concatenate([...], axis=0)에서 axis=0은 어떤 축(방향)으로 배열을 이어붙일 것인지를 지정하는 옵션

- axis=0 → 행(row) 방향으로 붙인다

- 행의 개수가 늘어나고, 열은 그대로 유지됨

- axis=1: 좌/우로 붙임 (열 늘어남)

In [9]:
np.concatenate([grid, grid],axis=0)

array([[1, 2, 3],
       [4, 5, 6],
       [1, 2, 3],
       [4, 5, 6]])

In [10]:
# 두 번째 축을 따라 연결(0부터 시작되는 인덱스 방식) 
np.concatenate([grid, grid], axis=1) #column으로 합치다 

array([[1, 2, 3, 1, 2, 3],
       [4, 5, 6, 4, 5, 6]])

For working with arrays of mixed dimensions, it can be clearer to ** use the ``np.vstack`` (vertical stack) and ``np.hstack`` (horizontal stack) functions**

In [11]:
x = np.array([1, 2, 3])
grid = np.array([[9, 8, 7],
                 [6, 5, 4]])

# 배열을 수직적으로 쌓음 
np.vstack([x, grid])

array([[1, 2, 3],
       [9, 8, 7],
       [6, 5, 4]])

In [12]:
# 배열을 수평적으로 쌓음
y = np.array([[99],
              [99]])
np.hstack([grid, y])

array([[ 9,  8,  7, 99],
       [ 6,  5,  4, 99]])

Similary, ``np.dstack`` will stack arrays along the third axis.

### Splitting of arrays

The opposite of concatenation is splitting, which is implemented by the functions ``np.split``, ``np.hsplit``, and ``np.vsplit``.  For each of these, we can pass a list of indices giving the split points:

In [15]:
x = [1, 2, 3, 99, 99, 3, 2, 1]
x1, x2, x3,x4 = np.split(x, [1,3, 5])
print(x1, x2, x3,x4)

[1] [2 3] [99 99] [3 2 1]


Notice that *N* split-points, leads to *N + 1* subarrays.
The related functions ``np.hsplit`` and ``np.vsplit`` are similar:

In [21]:
grid = np.arange(16).reshape((4, 4))
grid

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11],
       [12, 13, 14, 15]])

In [20]:
grid

array([[9, 8, 7],
       [6, 5, 4]])

In [22]:
upper, lower = np.vsplit(grid, [2])
print(upper)
print(lower)

[[0 1 2 3]
 [4 5 6 7]]
[[ 8  9 10 11]
 [12 13 14 15]]


In [53]:
left, right = np.hsplit(grid, [2])
print(left)
print(right)

[[ 0  1]
 [ 4  5]
 [ 8  9]
 [12 13]]
[[ 2  3]
 [ 6  7]
 [10 11]
 [14 15]]


Similarly, ``np.dsplit`` will split arrays along the third axis.

<!--NAVIGATION-->
< [Understanding Data Types in Python](02.01-Understanding-Data-Types.ipynb) | [Contents](Index.ipynb) | [Computation on NumPy Arrays: Universal Functions](02.03-Computation-on-arrays-ufuncs.ipynb) >

모델 학습에 사용되는 특성(feature)을 (N, 1)로 하고 목표 값(target)을 (N,)으로 구성하는 이유는 데이터 구조가 모델 입력과 출력의 요구사항에 맞도록 적합하게 설정

- 특성은 2차원 배열로, 목표 값은 1차원 배열로 구성해 일관된 데이터 처리와 수학적 연산의 편리함을 유지.

In [ ]:
from sklearn.linear_model import LinearRegression
import numpy as np

# 임의 데이터 생성
N = 10
X = np.random.rand(N, 1)  # (N, 1) 형태의 2차원 배열
y = 3 * X.ravel() + 2 + np.random.randn(N) * 0.1  # (N,) 형태의 1차원 배열
### X.ravel()은 (N,1)을 (N,)로 변환 1차원으로 변환
# 모델 초기화 및 학습
model = LinearRegression()
model.fit(X, y)

# 예측
y_pred = model.predict(X)


In [2]:
X

array([[0.73613843],
       [0.50815634],
       [0.58765119],
       [0.61958882],
       [0.91672011],
       [0.71052193],
       [0.36547981],
       [0.47249113],
       [0.4986792 ],
       [0.26277717]])

In [3]:
y

array([4.1989358 , 3.56522979, 3.73608323, 3.99871055, 4.79906523,
       4.194128  , 3.12456489, 3.40448707, 3.44657535, 2.92236572])

특성 X는 (N, 1) 형태:
선형 회귀 모델에서 입력 데이터는 (샘플 수, 특성 수) 형태로 요구됩니다. 즉, 각 데이터 포인트가 2차원 배열로 표현됩니다.
예를 들어, X가 (N, 1)인 경우 N개의 샘플이 있고 각 샘플이 하나의 특성을 가집니다. 이렇게 하면 X에 여러 개의 특성이 있더라도 차원 수에 맞춰 손쉽게 추가할 수 있습니다.

목표 값 y는 (N,) 형태:
y는 각 데이터 포인트에 대한 단일 목표 값을 가지며, (N,) 형태의 1차원 배열로 정의되어 단순하고 효율적입니다.
y가 1차원 배열이기 때문에 각 샘플에 정확히 하나의 목표 값이 매핑되며, 연산이 효율적입니다. 예측할 때도 모델은 (N,)의 형태로 목표 값을 반환합니다.

이렇게 구성하면, 특성 배열과 목표 값 배열을 사용해 모델 학습이 원활하게 이루어집니다. 특성 배열은 모델에 다차원 데이터를 입력할 수 있도록 구성되고, 목표 값은 단일 차원으로 구성되어 각 데이터 포인트의 예측 결과가 간단히 매칭될 수 있습니다.